<a href="https://colab.research.google.com/github/nicolaslepagecnam/REALIST-project/blob/main/realist_lightning_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!ip install torch
!pip install tfrecord

!pip install torch-geometric
!pip install functools

!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Object "install" is unknown, try "ip help".
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 KB 2.3 MB/s eta 0:00:00
  Created wheel for tfrecord: filename=tfrecord-1.14.1-py3-none-any.whl size=15653 sha256=743507d6cce82c122934a9961dfe83ecc014266d12c390faa6001970d8d04506
  Stored in directory: /root/.cache/pip/wheels/7d/d3/bb/80b8c03f1ce618f07f1fc58622ca540fdfa413e2c67bba0889
Successfully built tfrecord
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 29.9 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
  error: subprocess-exited-with-error
  
  × Running setup.py install for functools did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for functools ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> functools

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
Looking in

In [ ]:
# !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
# !pip install torch-sparse==0.6.11 -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html

import torch
# import torch_geometric
import pytorch_lightning as pl
import os

# print(torch.__version__)
# print(torch_geometric.__version__)

In [ ]:
if not os.path.exists("/content/test.tfrecord") :
  !wget "https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/test.tfrecord"
if not os.path.exists("/content/train.tfrecord") :
  !wget "https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/train.tfrecord"
if not os.path.exists("/content/meta.json") :
  !wget "https://storage.googleapis.com/dm-meshgraphnets/cylinder_flow/meta.json"

In [ ]:

import torch
from torch_geometric.data import Dataset, download_url

import tensorflow as tf
import json
import os
import functools
import numpy as np
import torch
# import torch_geometric_temporal as pygt
import torch_geometric as pyg

def _parse(proto, meta):
  """Parses a trajectory from tf.Example."""
  feature_lists = {k: tf.io.VarLenFeature(tf.string)
                   for k in meta['field_names']}
  features = tf.io.parse_single_example(proto, feature_lists)
  out = {}
  for key, field in meta['features'].items():
    data = tf.io.decode_raw(features[key].values, getattr(tf, field['dtype']))
    data = tf.reshape(data, field['shape'])
    if field['type'] == 'static':
      data = tf.tile(data, [meta['trajectory_length'], 1, 1])
    elif field['type'] == 'dynamic_varlen':
      length = tf.io.decode_raw(features['length_'+key].values, tf.int32)
      length = tf.reshape(length, [-1])
      data = tf.RaggedTensor.from_row_lengths(data, row_lengths=length)
    elif field['type'] != 'dynamic':
      raise ValueError('invalid data format')
    out[key] = data
  return out

def load_dataset(path, split):
  """Load dataset."""
  with open(os.path.join(path, 'meta.json'), 'r') as fp:
    meta = json.loads(fp.read())
  ds = tf.data.TFRecordDataset(os.path.join(path, split+'.tfrecord'))
  ds = ds.map(functools.partial(_parse, meta=meta), num_parallel_calls=8)
  ds = ds.prefetch(1)
  return ds

def edges_creator(connect):
  edges = np.zeros((2,1))

  for i in connect :
      for j in i :
          for k in i :

              if k != j :

                  temp_edge = np.zeros((2,1))
                  temp_edge[0] = j
                  temp_edge[1] = k

                  temp_list = np.argwhere((edges[0,:]-temp_edge[0])==0)

                  flag = 0

                  for l in temp_list :
                      if edges[1,int(l)] == temp_edge[1] :

                          flag += 1
                          break

                  if flag == 0:

                      edges = np.concatenate((edges,temp_edge), axis = 1)
  return edges

def add_targets(ds, fields, add_history):
  """Adds target and optionally history fields to dataframe."""
  def fn(trajectory):
    out = {}
    for key, val in trajectory.items():
      out[key] = val[1:-1]
      if key in fields:
        if add_history:
          out['prev|'+key] = val[0:-2]
        out['target|'+key] = val[2:]
    return out
  return ds.map(fn, num_parallel_calls=1)

def split_and_preprocess(ds, noise_field, noise_scale, noise_gamma):
  """Splits trajectories into frames, and adds training noise."""
  def add_noise(frame):
    noise = tf.random.normal(tf.shape(frame[noise_field]),
                             stddev=noise_scale, dtype=tf.float32)
    # don't apply noise to boundary nodes
    mask = tf.equal(frame['node_type'], 0)[:, 0]
    noise = tf.where(mask, noise, tf.zeros_like(noise))
    frame[noise_field] += noise
    frame['target|'+noise_field] += (1.0 - noise_gamma) * noise
    return frame

  ds = ds.flat_map(tf.data.Dataset.from_tensor_slices)
  ds = ds.map(add_noise, num_parallel_calls=1)
  ds = ds.shuffle(10000)
  ds = ds.repeat(None)
  return ds.prefetch(10)

params = {
    'cfd': dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                size=2, batch=2)
}

In [ ]:


# batches = []

# for i in range(len(dataset[0])) :
#   new_temp_list = []
#   for j in range(len(dataset)) :
#     new_temp_list.append(dataset[j][i])
#     print(dataset[j][i])

#   batch = pyg.data.Batch.from_data_list(new_temp_list)

#   batches.append(batch)

import os.path as osp

import math

import torch
from torch_geometric.data import Dataset, download_url


class Test_out_of_memory_dataset(torch_geometric.data.Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)


    @property
    def raw_file_names(self):
        return "/content/test.tfrecord"

    @property
    def processed_file_names(self):

        processed_data_list = []

        len =  math.ceil(100/32)

        for i in range(len) :
          processed_data_list.append(f'data_{i}.pt')

        # print(processed_data_list)
        return processed_data_list

    def download(self):
      pass

    def process(self):

        total_counter = 0
        idx = 0

        # counter = 0

        ds = load_dataset("/content", 'test') 
        # inputs = iter(ds)

        # for i in inputs :

        edges_list= []
        edge_w_list = []
        features_list = []
        y_list = []

        dataset = []

        cells_list = []
        size_list = []

        for i in ds :

          cells_list.append(i["cells"][0].numpy())

          edges = edges_creator(i["cells"][0].numpy())

          features  =  np.concatenate((i["velocity"].numpy(), i["pressure"].numpy(), i["mesh_pos"].numpy(), i["node_type"].numpy()), axis =-1)

          features = list(features)[:300]

          # print(features[0].shape[0])

          size_list.append(features[0].shape[0])

          data_list = []

          for j in features :

            # print(j.shape)

            data_list.append(pyg.data.Data(x = torch.tensor(j, dtype=torch.float), edge_index = torch.tensor(edges, dtype=torch.int64))) # torch.int32

            del j
          
          del features

          del edges

          dataset.append(data_list)

          del i

          del data_list

          print(len(dataset))
          print(len(size_list))
          print(len(cells_list))

          print(total_counter)
          
          if len(dataset) == 32 or total_counter == 99:

            print("entered saving loop")

            batches = []

            for i in range(len(dataset[0])) :
              new_temp_list = []
              for j in range(len(dataset)) :
                new_temp_list.append(dataset[j][i])

              batch = pyg.data.Batch.from_data_list(new_temp_list)
              batches.append(batch)

            torch.save([cells_list, size_list, batches], osp.join(self.processed_dir, f'data_{idx}.pt'))

            dataset = []
            cells_list = []
            size_list = []

            idx += 1
          
          total_counter +=1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

class Train_out_of_memory_dataset(torch_geometric.data.Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)


    @property
    def raw_file_names(self):
        return "/content/train.tfrecord"

    @property
    def processed_file_names(self):

        processed_data_list = []

        len =  math.ceil(1000/32)

        for i in range(len) :
          processed_data_list.append(f'data_{i}.pt')

        # print(processed_data_list)
        return processed_data_list

    def download(self):
      pass

    def process(self):

        total_counter = 0
        idx = 0

        # counter = 0

        ds = load_dataset("/content", 'train') 
        # inputs = iter(ds)

        # for i in inputs :

        edges_list= []
        edge_w_list = []
        features_list = []
        y_list = []

        dataset = []

        cells_list = []
        size_list = []

        for i in ds :

          cells_list.append(i["cells"][0].numpy())

          edges = edges_creator(i["cells"][0].numpy())

          features  =  np.concatenate((i["velocity"].numpy(), i["pressure"].numpy(), i["mesh_pos"].numpy(), i["node_type"].numpy()), axis =-1)

          features = list(features)[:300]

          # print(features[0].shape[0])

          size_list.append(features[0].shape[0])

          data_list = []

          for j in features :

            # print(j.shape)

            data_list.append(pyg.data.Data(x = torch.tensor(j, dtype=torch.float), 
                                           edge_index = torch.tensor(edges, dtype=torch.int64), 
                                           max_size = features[0].shape[0], 
                                           cells = i["cells"][0].numpy())) # torch.int32

            del j
          
          del features

          del edges

          dataset.append(data_list)

          del i

          del data_list

          print(len(dataset))
          print(len(size_list))
          print(len(cells_list))


          print(total_counter)

          if len(dataset) == 32 or total_counter == 999:

            print("entered saving loop")

            batches = []

            for i in range(len(dataset[0])) :
              new_temp_list = []
              for j in range(len(dataset)) :
                new_temp_list.append(dataset[j][i])

              batch = pyg.data.Batch.from_data_list(new_temp_list)
              batches.append(batch)

            torch.save(batches, osp.join(self.processed_dir, f'data_{idx}.pt'))

            dataset = []
            cells_list = []
            size_list = []

            idx += 1
          
          total_counter +=1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

# test_data = Test_out_of_memory_dataset(root = "/content/data")
# print(len(test_data))
# x = next(iter(test_data))
# print(x[0])
# print(len(x))

# train_data = Train_out_of_memory_dataset(root = "/content/data_train")
# print(len(train_data))
# x = next(iter(test_data))
# print(x[0])
# print(len(x))

In [ ]:
!pip install torchdiffeq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class light_datamodule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "/content/data_module", batch_size: int = 1):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = 1

    def setup(self, stage: str):
        self.train_data = torch_geometric.data.DataLoader(Train_out_of_memory_dataset(root = "/content/data_train"), batch_size=self.batch_size)
        self.val_data = torch_geometric.data.DataLoader(Test_out_of_memory_dataset(root = "/content/data"), batch_size=self.batch_size)

    def train_dataloader(self):
        return self.train_data

    def val_dataloader(self):
        return self.val_data

    def test_dataloader(self):
        return self.val_data

In [ ]:
from torchdiffeq import odeint_adjoint, odeint

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import ChebConv, GATv2Conv, SAGEConv
from torch_geometric.nn import global_mean_pool

from matplotlib import tri as mtri
import matplotlib.pyplot as plt

import random as rd


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # self.conv1 = ChebConv(6, hidden_channels, K = 2)
        # self.conv2 = ChebConv(hidden_channels, hidden_channels, K = 2)
        # self.conv3 = ChebConv(hidden_channels, hidden_channels, K = 2)
        # self.conv6 = ChebConv(hidden_channels, 3, K = 2)
        self.conv1 = GATv2Conv(6, hidden_channels, heads = heads)
        self.conv2 = GATv2Conv(hidden_channels*heads, hidden_channels, heads = heads)
        self.conv3 = GATv2Conv(hidden_channels*heads, hidden_channels, heads = heads)
        self.conv6 = GATv2Conv(hidden_channels*heads, 3, heads = 1)

    def forward(self, t, x):
      
        # print(t)

        # print(x.size())

        # edge_index = edge_index.long()

        # (x, edge_index) = input
        # 1. Obtain node embeddings 
        x = self.conv1(x, self.edge_index)
        x = x.relu()
        # x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv2(x, self.edge_index)
        x = x.relu()
        # x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv3(x, self.edge_index)
        x = x.relu()
        # x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv6(x, self.edge_index)

        x = torch.cat((x,torch.empty(x.size()).to("cuda")),dim = 1)

        return x
       
class Forecaster(torch.nn.Module):
    def __init__(self, method='euler', options=None):
        super().__init__()

        self.model_aug = GCN(hidden_channels=16, heads = 8)
        print(pyg.profile.count_parameters(self.model_aug))

        self.method = method
        self.options = options
        self.int_ = odeint_adjoint
        
    def forward(self, y0, t):
        

        (x, edge_index) = y0

        self.model_aug.edge_index = edge_index # .long()

        res = self.int_(self.model_aug, y0=x, t=t.to("cuda"), method=self.method, options=self.options)

        return res 

class gcn_lightning(pl.LightningModule):
    
    def __init__(self, lr=5e-4):
      super().__init__()

      self.full_model = Forecaster()
      self.criterion = torch.nn.MSELoss(reduction = "mean")

      self.lr = lr

    def training_step(self, batch, batch_idx):

      # start  = rd.randint(0,199)

      # stop = min(299, rd.randint(start+1,1+start+9*(self.current_epoch+1)))

      start  = 0

      stop = 299

      print("start = " + str(start) + ", stop = " + str(stop))

      skips = 1

      loss = 0

      t = torch.linspace(start*0.01, stop*0.01, steps=stop-start+1)

      out = self.full_model(y0 = (batch[start].x, batch[start].edge_index), t= t[::skips])

      for j in range(0,out.size(0),skips):

        index = start+j

        loss += (self.criterion(out[j,:,0:3], batch[index].x[:,0:3])/len(batch[::skips]))  # Compute the loss.
      
      return loss

    def validation_step(self, batch, batch_idx):

      print("testing step")

      loss = 0

      t = torch.linspace(0, 2.99, steps=300)

      out = self.full_model(y0 = (batch[0].x, batch[0].edge_index), t= t)

      for j in range(0,out.size(0)):

        loss += (self.criterion(out[j,:,0:3], batch[j].x[:,0:3])/len(batch))  # Compute the loss.
      
      print(loss)

      debut = 0
      fin = batch[0].max_size

      triang = mtri.Triangulation(batch[0].x.detach().numpy()[debut:fin,3], batch[0].x.detach().numpy()[debut:fin,4], batch[0].cells)

      fig, (ax1,ax2,ax3) = plt.subplots(3, 1, figsize=(15, 20))

      norm_x = np.sqrt(out[-1].detach().numpy()[debut:fin,0]**2 + out[-1].detach().numpy()[debut:fin,1]**2)
      norm_y = np.sqrt(batch[-1].x.detach().numpy()[debut:fin,0]**2 + batch[-1].x.detach().numpy()[debut:fin,1]**2)

      error = (norm_x-norm_y)**2

      ax1.tripcolor(triang, norm_y,  shading='gouraud' ) # x-velocity
      ax1.triplot(triang, 'ko-', ms=0.5, lw=0.3)

      ax2.tripcolor(triang, norm_x,  shading='gouraud' ) # x-velocity
      ax2.triplot(triang, 'ko-', ms=0.5, lw=0.3)

      ax3.tripcolor(triang, error,  shading='gouraud' ) # x-velocity
      ax3.triplot(triang, 'ko-', ms=0.5, lw=0.3)

      plt.show()
      
      return loss # Clear gradients.

    def configure_optimizers(self):

        optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)
        return optimizer

In [ ]:
torch.cuda.empty_cache()

data_module = light_datamodule()
test_model = gcn_lightning()

trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    # auto_lr_find=True,
    # logger=wandb_logger,    # W&B integration
    log_every_n_steps = 1,   # set the logging frequency
    check_val_every_n_epoch = 1,
    max_epochs= 200,
    deterministic=True
    # callbacks=[TQDMProgressBar(refresh_rate=1),checkpoint_callback],
)

# lr_finder = trainer.tuner.lr_find(test_model, data_module)

# # Results can be found in
# lr_finder.results

# # Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

trainer.fit(test_model , data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


69388


Processing...


1
1
1
0
2
2
2
1
3
3
3
2
4
4
4
3
5
5
5
4
6
6
6
5
7
7
7
6
8
8
8
7
9
9
9
8
10
10
10
9
11
11
11
10
12
12
12
11
13
13
13
12
14
14
14
13
15
15
15
14
16
16
16
15
17
17
17
16
18
18
18
17
19
19
19
18
20
20
20
19
21
21
21
20
22
22
22
21
23
23
23
22
24
24
24
23
25
25
25
24
26
26
26
25
27
27
27
26
28
28
28
27
29
29
29
28
30
30
30
29
31
31
31
30
32
32
32
31
entered saving loop
1
1
1
32
2
2
2
33
3
3
3
34
4
4
4
35
5
5
5
36
6
6
6
37
7
7
7
38
8
8
8
39
9
9
9
40
10
10
10
41
11
11
11
42
12
12
12
43
13
13
13
44
14
14
14
45
15
15
15
46
16
16
16
47
17
17
17
48
18
18
18
49
19
19
19
50
20
20
20
51
21
21
21
52
22
22
22
53
23
23
23
54
24
24
24
55
25
25
25
56
26
26
26
57
27
27
27
58
28
28
28
59
29
29
29
60
30
30
30
61
31
31
31
62
32
32
32
63
entered saving loop
1
1
1
64
2
2
2
65
3
3
3
66
4
4
4
67
5
5
5
68
6
6
6
69
7
7
7
70
8
8
8
71
9
9
9
72
10
10
10
73
11
11
11
74
12
12
12
75
13
13
13
76
14
14
14
77
15
15
15
78
16
16
16
79
17
17
17
80
18
18
18
81
19
19
19
82
20
20
20
83
21
21
21
84
22
22
22
85
23
23
23
86
24
24
2

Done!
/usr/local/lib/python3.8/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Processing...


1
1
1
0
2
2
2
1
3
3
3
2
4
4
4
3
5
5
5
4
6
6
6
5
7
7
7
6
8
8
8
7
9
9
9
8
10
10
10
9
11
11
11
10
12
12
12
11
13
13
13
12
14
14
14
13
15
15
15
14
16
16
16
15
17
17
17
16
18
18
18
17
19
19
19
18
20
20
20
19
21
21
21
20
22
22
22
21
23
23
23
22
24
24
24
23
25
25
25
24
26
26
26
25
27
27
27
26
28
28
28
27
29
29
29
28
30
30
30
29
31
31
31
30
32
32
32
31
entered saving loop
1
1
1
32
2
2
2
33
3
3
3
34
4
4
4
35
5
5
5
36
6
6
6
37
7
7
7
38
8
8
8
39
9
9
9
40
10
10
10
41
11
11
11
42
12
12
12
43
13
13
13
44
14
14
14
45
15
15
15
46
16
16
16
47
17
17
17
48
18
18
18
49
19
19
19
50
20
20
20
51
21
21
21
52
22
22
22
53
23
23
23
54
24
24
24
55
25
25
25
56
26
26
26
57
27
27
27
58
28
28
28
59
29
29
29
60
30
30
30
61
31
31
31
62
32
32
32
63
entered saving loop
1
1
1
64
2
2
2
65
3
3
3
66
4
4
4
67
5
5
5
68
6
6
6
69
7
7
7
70
8
8
8
71
9
9
9
72
10
10
10
73
11
11
11
74
12
12
12
75
13
13
13
76
14
14
14
77
15
15
15
78
16
16
16
79
17
17
17
80
18
18
18
81
19
19
19
82
20
20
20
83
21
21
21
84
22
22
22
85
23
23
23
86
24
24
2

Done!
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | full_model | Forecaster | 69.4 K
1 | criterion  | MSELoss    | 0     
------------------------------------------
69.4 K    Trainable params
0         Non-trainable params
69.4 K    Total params
0.278     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: ignored